In [ ]:

import sys
import os
import logging
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency, ttest_ind_from_stats

# Ajouter le chemin du dossier src
sys.path.append(os.path.abspath("../src"))

# Importer la configuration
from config import PROCESSED_DATA_PATH

# Configuration du logger
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)
    

In [ ]:

# Fonction pour charger les données
def load_data(filepath, name):
    """Charge un fichier CSV et affiche ses informations de base"""
    if os.path.exists(filepath):
        try:
            df = pd.read_csv(filepath)
            logger.info(f"{name}.csv chargé avec succès ({df.shape[0]} lignes, {df.shape[1]} colonnes) !")
            logger.info(f"Aperçu des 5 premières lignes :\n{df.head()}")
            return df
        except Exception as e:
            logger.error(f"Erreur lors du chargement de {name}.csv : {e}")
    else:
        logger.error(f"Fichier {name}.csv introuvable !")
    return None
    

In [ ]:

# Dictionnaire des fichiers de données
datasets = {
    "events": os.path.join(PROCESSED_DATA_PATH, "events_processed.csv"),
    "category_tree": os.path.join(PROCESSED_DATA_PATH, "category_tree_processed.csv"),
    "item_properties": os.path.join(PROCESSED_DATA_PATH, "item_properties_processed.csv")
}

# Chargement des fichiers
events_df = load_data(datasets["events"], "events")
category_tree_df = load_data(datasets["category_tree"], "category_tree")
item_properties_df = load_data(datasets["item_properties"], "item_properties")
    

In [ ]:

# Analyse des événements utilisateurs
if events_df is not None:
    logger.info("\nAnalyse des événements utilisateurs :")
    logger.info(f"Types d'événements uniques : {events_df['event'].unique()}")
    logger.info(f"Nombre d'événements par type :\n{events_df['event'].value_counts()}")

    # Test A/B - Comparaison des taux de conversion
    conversion_rates = events_df.groupby("event")["visitorid"].nunique()
    view_count = conversion_rates.get("view", 0)
    addtocart_count = conversion_rates.get("addtocart", 0)

    if view_count > 0 and addtocart_count > 0:
        t_stat, p_value = ttest_ind_from_stats(
            mean1=view_count, std1=np.sqrt(view_count), nobs1=view_count,
            mean2=addtocart_count, std2=np.sqrt(addtocart_count), nobs2=addtocart_count
        )
        logger.info(f"Test A/B - Taux de conversion : t_stat={t_stat:.4f}, p_value={p_value:.4f}")
    

In [ ]:

# Analyse de la hiérarchie des catégories
if category_tree_df is not None:
    logger.info("\nAnalyse de la hiérarchie des catégories :")
    nb_categories = category_tree_df["categoryid"].nunique()
    nb_root_categories = category_tree_df["parentid"].isna().sum()
    logger.info(f"Nombre total de catégories : {nb_categories}")
    logger.info(f"Nombre de catégories racines (sans parent) : {nb_root_categories}")

    # Test A/B - Comparaison des niveaux de catégories
    root_categories = category_tree_df[category_tree_df["parentid"].isna()].shape[0]
    sub_categories = category_tree_df[category_tree_df["parentid"].notna()].shape[0]

    chi2, p_value = chi2_contingency([[root_categories, sub_categories]])[:2]
    logger.info(f"Test A/B - Répartition des catégories : chi2={chi2:.4f}, p_value={p_value:.4f}")
    

In [ ]:

# Analyse des propriétés des articles
if item_properties_df is not None:
    logger.info("\nAnalyse des propriétés des articles :")
    logger.info(f"Nombre total d'articles : {item_properties_df['itemid'].nunique()}")
    logger.info(f"Nombre total de propriétés uniques : {item_properties_df['property'].nunique()}")

    # Test A/B - Disponibilité des articles
    available_data = item_properties_df[item_properties_df['property'] == 'available']
    if not available_data.empty:
        available_counts = available_data["value"].value_counts()
        if 1 in available_counts and 0 in available_counts:
            chi2, p_value = chi2_contingency([[available_counts[1], available_counts[0]]])[:2]
            logger.info(f"Test A/B - Disponibilité des articles : chi2={chi2:.4f}, p_value={p_value:.4f}")
    